In [15]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Concatenate
)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

# --- Load and Preprocess Text Data ---
file_path = r"F:\\irmatext\\Hurricane_irma.csv"
data = pd.read_csv(file_path)
data['processed_data'] = data['processed_data'].fillna('').astype(str)
data['Binary_Class'] = data['Binary_Class'].map({'Non-Damage': 0, 'Damage': 1})

sentences = [sentence.split() for sentence in data['processed_data']]
labels = data['Binary_Class'].values

X_train_text, X_test_text, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=42)

# Train Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1)
vocab_size = len(word2vec_model.wv)
embedding_dim = word2vec_model.vector_size

embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))
word_index = {word: idx + 1 for idx, word in enumerate(word2vec_model.wv.index_to_key)}
for word, idx in word_index.items():
    embedding_matrix[idx] = word2vec_model.wv[word]

# Convert text data to padded sequences
X_train_text = [[word_index.get(word, 0) for word in sentence] for sentence in X_train_text]
X_test_text = [[word_index.get(word, 0) for word in sentence] for sentence in X_test_text]
X_train_text = pad_sequences(X_train_text, maxlen=100)
X_test_text = pad_sequences(X_test_text, maxlen=100)

# --- Preprocess Image Data ---
train_data_dir = r"F:\\data_image\\preprocessed_images\\train"
val_data_dir = r"F:\\data_image\\preprocessed_images\\validation"

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(224, 224), batch_size=16, class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_data_dir, target_size=(224, 224), batch_size=16, class_mode='binary'
)

# --- Build Text Model ---
text_input = Input(shape=(100,), name="text_input")
text_embedding = Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim, weights=[embedding_matrix],
                           input_length=100, trainable=True)(text_input)
text_cnn = Conv1D(filters=128, kernel_size=3, activation='relu')(text_embedding)
text_cnn = MaxPooling1D(pool_size=2)(text_cnn)
text_cnn = Flatten()(text_cnn)
text_output = Dense(128, activation='relu')(text_cnn)

# --- Build Image Model ---
image_input = Input(shape=(224, 224, 3), name="image_input")
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=image_input)
for layer in base_model.layers[:-4]:  # Unfreeze last 4 layers
    layer.trainable = False
image_cnn = Flatten()(base_model.output)
image_output = Dense(256, activation='relu')(image_cnn)

# --- Merge Models ---
merged = Concatenate()([text_output, image_output])
merged_dense = Dense(128, activation='relu')(merged)
merged_output = Dense(1, activation='sigmoid', name="output")(merged_dense)

model = Model(inputs=[text_input, image_input], outputs=merged_output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# --- Define Data Generator ---
def data_generator(text_data, image_generator, labels):
    while True:
        image_batch, _ = next(image_generator)  # Get images and ignore labels from the generator
        batch_size = len(image_batch)
        start_idx = np.random.randint(0, len(text_data) - batch_size + 1)
        text_batch = text_data[start_idx:start_idx + batch_size]
        label_batch = labels[start_idx:start_idx + batch_size]
        yield {"text_input": text_batch, "image_input": image_batch}, label_batch

train_gen = data_generator(X_train_text, train_generator, y_train)
val_gen = data_generator(X_test_text, val_generator, y_test)

steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

# --- Train Model ---
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

# --- Predict with Validation Generator ---
def predict_generator(text_data, image_generator):
    for _ in range(len(image_generator)):
        image_batch, _ = next(image_generator)
        batch_size = len(image_batch)
        start_idx = np.random.randint(0, len(text_data) - batch_size + 1)
        text_batch = text_data[start_idx:start_idx + batch_size]
        yield {"text_input": text_batch, "image_input": image_batch}

y_pred_probs = model.predict(predict_generator(X_test_text, val_generator), steps=validation_steps)
y_pred_classes = (y_pred_probs > 0.5).astype("int32")

# Align y_test
y_test_eval = y_test[:validation_steps * train_generator.batch_size]

# --- Evaluate Model ---
print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy_score(y_test_eval, y_pred_classes):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_eval, y_pred_classes))

Found 4379 images belonging to 8 classes.
Found 1713 images belonging to 8 classes.


E:\Anaconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3458s 13s/step - accuracy: 0.8702 - loss: 0.4771 - val_accuracy: 0.8745 - val_loss: 0.3666 - learning_rate: 0.0010
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3453s 13s/step - accuracy: 0.9306 - loss: 0.1950 - val_accuracy: 0.8646 - val_loss: 0.3999 - learning_rate: 0.0010
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3456s 13s/step - accuracy: 0.9745 - loss: 0.0813 - val_accuracy: 0.8466 - val_loss: 0.6725 - learning_rate: 0.0010
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3458s 13s/step - accuracy: 0.9951 - loss: 0.0172 - val_accuracy: 0.8459 - val_loss: 0.8127 - learning_rate: 0.0010
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3467s 13s/step - accuracy: 0.9972 - loss: 0.0124 - val_accuracy: 0.8506 - val_loss: 0.8003 - learning_rate: 5.0000e-04
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 3451s 13s/step - accuracy: 0.9996 - loss: 0.0029 - val_accuracy: 0.8581 - val_loss: 0.8861 - learning_rate: 5.0000e-04


ValueError: When passing a Python generator to a Keras model, the generator must return a tuple, either (input,) or (inputs, targets) or (inputs, targets, sample_weights). Received: {'text_input': array([[   0,    0,    0, ..., 6226,    2,   91],
       [   0,    0,    0, ..., 4129,   31, 1280],
       [   0,    0,    0, ..., 2337,  362,   22],
       ...,
       [   0,    0,    0, ..., 2058, 1142,  312],
       [   0,    0,    0, ..., 1936,    6,   42],
       [   0,    0,    0, ...,    2,    1, 3687]]), 'image_input': array([[[[0.7176471 , 0.6509804 , 0.72156864],
         [0.7176471 , 0.6509804 , 0.72156864],
         [0.7176471 , 0.6509804 , 0.7137255 ],
         ...,
         [0.4039216 , 0.20784315, 0.07843138],
         [0.3921569 , 0.19607845, 0.06666667],
         [0.37647063, 0.1764706 , 0.05490196]],

        [[0.69803923, 0.6313726 , 0.7019608 ],
         [0.7019608 , 0.63529414, 0.7058824 ],
         [0.7019608 , 0.63529414, 0.69803923],
         ...,
         [0.4039216 ,